In [8]:
import os
import time

import sys
sys.path.append('..') # to import local EnhancedDataSynthesizer version

import matplotlib.pyplot as plt
plt.style.use('classic')

from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.lib.utils import display_bayesian_network

from DataSynthesizer.lib.utils import mutual_information

from DataSynthesizer.federated_utils.evaluation import TaskUtility, HistogramAndHeatmapComparison


import numpy as np
import pandas as pd
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

file_path=os.getcwd() 
traindata = os.path.join(file_path+'/data/',"adult_complete.csv")
testdata = os.path.join(file_path+'/data/',"adult_test.csv")

data=traindata
diff=0
k=2
sensi=None 
#sensi='relationship'
target='income'


In [9]:
#output_dir = 'out/correlated_attribute_mode_ga/'
output_dir = 'out/correlated_attribute_mode/'

description_file_ga = os.path.join(file_path, output_dir, "ga_corrmeta.json")

if sensi:
    synthetic_data = os.path.join(file_path, output_dir, "DSv_custom_"+str(k)+"_syntraindata.csv")
else:
    synthetic_data = os.path.join(file_path, output_dir, "DSv_"+str(k)+"_syntraindata.csv")

input_df = pd.read_csv(data, skipinitialspace=True)
test_df = pd.read_csv(testdata, skipinitialspace=True)

# An attribute is categorical if its domain size is less than this threshold.
# Here modify the threshold to adapt to the domain size of "education" (which is 14 in input dataset).
threshold_value = 20

# Specify categorical attributes
categorical_attributes = {'workclass': True, 'education-num': True, 'marital-status': True, 'occupation': True, 'relationship': True,
                          'race': True, 'sex': True, 'native-country': True, 'income': True}

# Specify which attributes are candidate keys of input dataset.
candidate_keys = {}

# A parameter in Differential Privacy. It roughly means that changing a row in the input dataset will not
# change the probability of getting the same output more than a multiplicative difference of exp(epsilon).
# Increase epsilon value to reduce the injected noises. Set epsilon=0 to turn off differential privacy.
epsilon = diff

# The maximum number of parents in Bayesian network, i.e., the maximum number of incoming edges.
degree_of_bayesian_network = k

# Number of tuples generated in synthetic dataset.
num_tuples_to_generate = len(input_df)

start=time.time()

#Data Describer:
describer_ga = DataDescriber(category_threshold=threshold_value)
#describer_ga.describe_dataset_in_correlated_attribute_mode_ga(dataset_file=data,
describer_ga.describe_dataset_in_correlated_attribute_mode(dataset_file=data,
                                                    epsilon=epsilon,
                                                    k=degree_of_bayesian_network,
                                                    attribute_to_is_categorical=categorical_attributes,
                                                    attribute_to_is_candidate_key=candidate_keys,
                                                    #source_genes=10,
                                                    #genepool_size=200,
                                                    #epochs=400,
                                                    #sensi=sensi,
                                                    #target=target
                                                    )

describer_ga.save_dataset_description_to_file(description_file_ga)
print("Wrote network to " + description_file_ga)

end=time.time()
bn_time=end-start
print("Runtime Data Description:", bn_time)

network=describer_ga.bayesian_network

fitness = 0
for (c, ps) in network:
        for p in ps:
            fitness += mutual_information(input_df[c], input_df[[p]])
print("Fitness:", fitness)

display_bayesian_network(describer_ga.bayesian_network)
start=time.time()

================ Constructing Bayesian Network (BN) ================
Adding ROOT race
Adding attribute native-country
Adding attribute education-num
Adding attribute occupation
Adding attribute workclass
Adding attribute age
Adding attribute marital-status
Adding attribute relationship
Adding attribute sex
Adding attribute hours-per-week
Adding attribute income
Adding attribute capital-gain
Adding attribute capital-loss
========================== BN constructed ==========================
Wrote network to c:\Users\DMartinez\Documents\Repositories\EnhancedDataSynthesizer\notebooks\out/correlated_attribute_mode/ga_corrmeta.json
Runtime Data Description: 259.8283190727234
Fitness: 3.2861418606177883
Constructed Bayesian network:
    native-country has parents ['race'].
    education-num  has parents ['native-country', 'race'].
    occupation     has parents ['education-num', 'native-country'].
    workclass      has parents ['occupation', 'education-num'].
    age            has parents ['

In [10]:
#Data Generator:
ga_generator = DataGenerator()
ga_generator.generate_dataset_in_correlated_attribute_mode(num_tuples_to_generate, description_file_ga)
ga_generator.save_synthetic_data(synthetic_data)

end=time.time()
gen_time=end-start
print("Runtime Data Generation:", gen_time)

Runtime Data Generation: 17.341266632080078


In [11]:
syn_df = pd.read_csv(synthetic_data)

In [12]:
# output = True
# if output:
#             textf = os.path.join(output_dir, 'k_'+str(degree_of_bayesian_network)+'_epochs_'+str(400)+'.txt')
#             with open(textf, 'w') as w:
#                 w.write("================"+'\n'+
#                         "Runtime Data Description: "+str(bn_time)+'\n'+
#                         "Fitness: "+str(fitness)+'\n'+
#                         "Runtime Data Generation: "+str(gen_time)+'\n'
#                         )

#             syn_df = pd.read_csv(synthetic_data)

#             # Label Encoding
#             header = list(input_df.columns.values)

#             for i in range(len(header)):

#                 if header[i] in ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']:
#                     input_df.iloc[:,i]=le.fit_transform(list(input_df.iloc[:,i])).astype(float)
#                     syn_df.iloc[:,i]=le.fit_transform(list(syn_df.iloc[:,i])).astype(float)

#             header= ['age', 'wcl', 'edu', 'mst', 'occ', 'rel', 'rac', 'sex', 'cga', 'clo', 'hpw', 'nat', 'inc']
#             correlations_syn = syn_df.corr()
#             fig = plt.figure()
#             ax = fig.add_subplot(1,1,1)
#             cax = ax.matshow(correlations_syn, cmap='seismic', vmin=-1, vmax=1)
#             ticks = np.arange(0,len(header),1)
#             ax.set_xticks(ticks)
#             ax.set_yticks(ticks)
#             ax.set_xticklabels(header)
#             ax.set_yticklabels(header)
#             #ax.set_xlabel("DataSynthesizer with \u03B5="+str(diff))
#             plt.savefig(output_dir+'heatsyn_custom_rel.png', bbox_inches='tight')
#             plt.show()

#             correlations_real = input_df.corr()
#             fig = plt.figure()
#             ax = fig.add_subplot(1,1,1)
#             cax = ax.matshow(correlations_real, cmap='seismic', vmin=-1, vmax=1)
#             fig.colorbar(cax)
#             ticks = np.arange(0,len(header),1)
#             ax.set_xticks(ticks)
#             ax.set_yticks(ticks)
#             ax.set_xticklabels(header)
#             ax.set_yticklabels(header)
#             ax.set_xlabel("Original")
#             plt.savefig(output_dir+'heatreal.png', bbox_inches='tight')
#             plt.show()

#             for i in range(len(header)):
#                 plt.hist(input_df.iloc[:,i], color='green', label='Original: '+header[i])
#                 plt.legend()
#                 plt.hist(syn_df.iloc[:,i],histtype='step', color='blue', label='DS with \u03B5='+str(diff)+': '+header[i])
#                 plt.legend()
#                 plt.savefig(output_dir+'histograms/'+header[i]+'.png', bbox_inches='tight')
#                 plt.show()

In [13]:
output=True
if output: 
    HistogramAndHeatmapComparison(data, synthetic_data, output_dir)
    TaskUtility(input_df,syn_df,test_df,target,output_dir,'Classification')

c:\Users\DMartinez\Documents\Repositories\EnhancedDataSynthesizer\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\DMartinez\Documents\Repositories\EnhancedDataSynthesizer\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options: